# Managing Threads and Messages in the OpenAI Assistants API

## Introduction

In this lesson, we'll explore how to effectively manage Threads and Messages in the OpenAI Assistants API. Threads and Messages are fundamental concepts that represent conversation sessions between an Assistant and a user. We'll cover creating threads, managing messages, handling attachments, and understanding context window management.


First, let's set up our environment:

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

In [ ]:
from openai import OpenAI
import time

# Initialize the OpenAI client
client = OpenAI()

## Understanding Threads and Messages

Threads represent conversation sessions and can contain up to 100,000 Messages. When the size of Messages exceeds the model's context window, the Thread automatically handles truncation, prioritizing the most relevant messages.

### Creating a Basic Thread

Let's start by creating a simple thread:

In [ ]:
def create_basic_thread():
    thread = client.beta.threads.create()
    print(f"Created thread with ID: {thread.id}")
    return thread

# Create a new thread
thread = create_basic_thread()

### Creating a Thread with Initial Messages

You can create a thread with initial messages:

In [ ]:
def create_thread_with_messages():
    thread = client.beta.threads.create(
        messages=[
            {
                "role": "user",
                "content": "I need help analyzing this dataset.",
            }
        ]
    )
    return thread

## Working with Message Content Types

Messages can contain different types of content, including text, images, and file attachments. Let's explore each type:

### Text Messages

In [ ]:
def add_text_message(thread_id, content):
    message = client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=content
    )
    return message

# Example usage
message = add_text_message(thread.id, "Can you help me understand this data?")

### Messages with File Attachments

In [ ]:
def create_message_with_attachment(thread_id, file_path):
    # First, upload the file
    file = client.files.create(
        file=open(file_path, "rb"),
        purpose="assistants"
    )
    
    # Create message with attachment
    message = client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content="Please analyze this file.",
        attachments=[
            {
                "file_id": file.id,
                "tools": [{"type": "code_interpreter"}]
            }
        ]
    )
    return message

### Messages with Images

In [ ]:
def create_message_with_image(thread_id, image_path):
    # Upload image file
    file = client.files.create(
        file=open(image_path, "rb"),
        purpose="vision"
    )
    
    # Create message with image
    message = client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=[
            {
                "type": "text",
                "text": "What do you see in this image?"
            },
            {
                "type": "image_file",
                "image_file": {"file_id": file.id}
            }
        ]
    )
    return message

## Managing Context Windows

The Assistants API provides ways to control how much context is used in conversations. Here's how to manage context windows:

In [ ]:
def create_run_with_context_limits(thread_id, assistant_id):
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
        # Limit the tokens used in the conversation
        max_prompt_tokens=500,
        max_completion_tokens=1000
    )
    return run

## Working with Message Annotations

Assistant messages may contain annotations for file citations or file paths. Here's how to process them:

In [ ]:
def process_message_annotations(thread_id, message_id):
    # Retrieve the message
    message = client.beta.threads.messages.retrieve(
        thread_id=thread_id,
        message_id=message_id
    )
    
    # Process the message content
    if not message.content or not message.content[0].text:
        return "No text content found"
    
    message_content = message.content[0].text
    annotations = message_content.annotations
    citations = []
    
    # Process annotations
    for index, annotation in enumerate(annotations):
        # Replace the text with a footnote
        message_content.value = message_content.value.replace(
            annotation.text, 
            f' [{index}]'
        )
        
        # Handle different annotation types
        if hasattr(annotation, 'file_citation'):
            cited_file = client.files.retrieve(annotation.file_citation.file_id)
            citations.append(
                f'[{index}] {annotation.file_citation.quote} from {cited_file.filename}'
            )
        elif hasattr(annotation, 'file_path'):
            cited_file = client.files.retrieve(annotation.file_path.file_id)
            citations.append(
                f'[{index}] File reference: {cited_file.filename}'
            )
    
    # Combine message and citations
    final_content = message_content.value
    if citations:
        final_content += '\n\nReferences:\n' + '\n'.join(citations)
    
    return final_content

## Best Practices

1. **Thread Management:**
   - Create new threads for distinct conversation sessions
   - Don't exceed 100,000 messages per thread
   - Clean up or archive old threads when they're no longer needed

2. **Message Content:**
   - Keep message content clear and focused
   - Use appropriate content types (text, images, files) based on the use case
   - Handle file attachments securely

3. **Context Window:**
   - Monitor token usage in long conversations
   - Use max_prompt_tokens and max_completion_tokens when needed
   - Consider using truncation strategies for long conversations

## Exercise

Try this exercise to practice working with threads and messages:

In [ ]:
def practice_thread_management():
    # 1. Create a new thread
    thread = client.beta.threads.create()
    
    # 2. Add a text message
    message1 = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content="Hello, I have some questions about data analysis."
    )
    
    # 3. Add a second message
    message2 = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content="Can you help me understand statistical significance?"
    )
    
    # 4. List all messages in the thread
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    
    # 5. Print the conversation
    for message in messages:
        print(f"Role: {message.role}")
        print(f"Content: {message.content[0].text.value}\n")
    
    return thread, messages

# Run the exercise
thread, messages = practice_thread_management()

## Conclusion

Understanding how to effectively manage threads and messages is crucial for building robust applications with the OpenAI Assistants API. Remember to handle context windows appropriately, process message annotations when needed, and follow best practices for thread management.

Try experimenting with different message types and content combinations to better understand how they work in your specific use case.